In [21]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import numpy as np
from nltk.stem import WordNetLemmatizer

In [22]:
df = pd.read_csv('/Users/justinchen/Downloads/BookDataset1.csv')
df = df.drop(['Image'], axis = 1)
df.replace(np.nan, '', regex=True, inplace = True)
df

,Date,Image,Title,Subtitle,Author,Transcript
0,1709,http://books.google.com/books/content?id=JHYPA...,The Golden Spy,"Or, a Political Journal of the British Nights ...",['Charles Gildon'],"Said iS as cee SS tttne Ta SR aa ds ,e : f ee ..."
1,1705,http://books.google.com/books/content?id=6SFhA...,The Whole Art of Dying. In Two Parts. The Firs...,,['ART.'],oe eee RY ape ar Se Oe. a 5363. a Pie SSar se ...
2,1707,http://books.google.com/books/content?id=0ahck...,"A Vindication of the Eternal Law, and Everlast...",,['John BEART'],aE A a ef oe sik Sy PS Uae Ne pansVINDICATI...
3,1701,http://books.google.com/books/content?id=KwEAA...,The Mysteries of Opium Reveal'd,"By Dr. John Jones, ...",['John Jones'],"ei, ps ey ee SS a ie Geeee aay 4RarBA Ea Nos7 ..."
4,1704,http://books.google.com/books/content?id=7rabk...,"A Collection of Voyages and Travels, Some Now ...",,,"7, ia Eee ene a ia I tk ae S . Boe a8 d AO:F :..."
...,...,...,...,...,...,...
145,1846,http://books.google.com/books/content?id=XKERA...,An American Dictionary of the English Language,"Exhibiting the Origin, Orthography, Pronunciat...",['Noah Webster'],"n Pa v,oa S orn lis C,Qolaate . ee a ocenmsp..."
146,1844,http://books.google.com/books/content?id=ff4YA...,An American Dictionary of the English Language,"Exhibiting the Origin, Orthography, Pronunciat...",['Noah Webster'],":j ten JZ. MLSIAN AMERICAN A, te OF THEENGLI..."
147,1845,http://books.google.com/books/content?id=RWhmA...,Memoirs of the Lady Hester Stanhope,As Related by Herself in Conversations with He...,['Lady Hester Stanhope'],"MEMOIRSOF THELADY HESTER STANHOPE,AS RELATED B..."
148,1847,http://books.google.com/books/content?id=CF9gA...,"Mary Anne Wellington, the soldier's daughter, ...",,['Richard COBBOLD'],".MARY ANNE WELLINGTON,THESOLDIERS DAUGHTER,WIF..."


In [23]:
def len_title(df):
    len_list = []
    for title in df['Title']:
        wordList = title.split()
        len_list.append(len(wordList))
    df['Title Length'] = len_list

def len_subtitle(df):
    len_list = []
    for title in df['Subtitle']:
        wordList = title.split()
        len_list.append(len(wordList))
    df['Subtitle Length'] = len_list
    
# combination of title and subtitle, but also includes publishing info, author, etc.
def len_cover(df):
    len_list = []
    for title in df['Transcript']:
        wordList = title.split()
        len_list.append(len(wordList))
    df['Words in Whole Cover'] = len_list

In [24]:
len_title(df)
len_subtitle(df)
len_cover(df)
df.head()

,Date,Image,Title,Subtitle,Author,Transcript,Title Length,Subtitle Length,Words in Whole Cover
0,1709,http://books.google.com/books/content?id=JHYPA...,The Golden Spy,"Or, a Political Journal of the British Nights ...",['Charles Gildon'],"Said iS as cee SS tttne Ta SR aa ds ,e : f ee ...",3,35,135
1,1705,http://books.google.com/books/content?id=6SFhA...,The Whole Art of Dying. In Two Parts. The Firs...,,['ART.'],oe eee RY ape ar Se Oe. a 5363. a Pie SSar se ...,71,0,220
2,1707,http://books.google.com/books/content?id=0ahck...,"A Vindication of the Eternal Law, and Everlast...",,['John BEART'],aE A a ef oe sik Sy PS Uae Ne pansVINDICATI...,10,0,86
3,1701,http://books.google.com/books/content?id=KwEAA...,The Mysteries of Opium Reveal'd,"By Dr. John Jones, ...",['John Jones'],"ei, ps ey ee SS a ie Geeee aay 4RarBA Ea Nos7 ...",5,5,171
4,1704,http://books.google.com/books/content?id=7rabk...,"A Collection of Voyages and Travels, Some Now ...",,,"7, ia Eee ene a ia I tk ae S . Boe a8 d AO:F :...",48,0,138


In [25]:
import re
#removing stopwords and urls, lemmatization, making lowercase 
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocessing(text, stem=False):
    text = str(text).lower().strip()
    tokens = []
    for token in text.split():
        #Substituting multiple spaces with single space
        token = re.sub(r'\s+', ' ', token, flags=re.I)
        if token not in stop_words:
            if stem:
                tokens.append(lemmatizer.lemmatize(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [26]:
from operator import itemgetter
def freq_words(df, N):
    word_dict = {}
    for title in df['Transcript']:
        wordList = title.lower().split()
        for word in wordList:
            if word in word_dict.keys():
                word_dict[word] += 1
            else:
                word_dict[word] = 1
    final_dict = dict(sorted(word_dict.items(), key = itemgetter(1), reverse=True)[:N])
    return final_dict

In [27]:
freq_words(df, 10)

{'the': 742,
 'of': 634,
 'and': 563,
 'a': 414,
 ':': 256,
 'in': 250,
 '.': 205,
 ';': 190,
 'to': 186,
 'ee': 162}

In [28]:
def freq_words_processed(df, N):
    df1 = df.copy()
    df1.Transcript = df1.Transcript.apply(lambda x: preprocessing(x))
    word_dict = {}
    for title in df1['Transcript']:
        wordList = title.split()
        for word in wordList:
            if word in word_dict.keys():
                word_dict[word] += 1
            else:
                word_dict[word] = 1
    final_dict = dict(sorted(word_dict.items(), key = itemgetter(1), reverse=True)[:N])
    return final_dict

In [29]:
freq_words_processed(df, 10)

{':': 256,
 '.': 205,
 ';': 190,
 'ee': 162,
 'ae': 152,
 '4': 100,
 ',': 74,
 'e': 73,
 '2': 68,
 'oe': 64}

In [30]:
from statistics import mean 
def avg_len_title(df):
    len_list = []
    for title in df['Title']:
        wordList = title.split()
        len_list.append(len(wordList))
    return mean(len_list)

def avg_len_subtitle(df):
    len_list = []
    for title in df['Subtitle']:
        wordList = title.split()
        len_list.append(len(wordList))
    return mean(len_list)
    
# combination of title and subtitle, but also includes publishing info, author, etc.
def avg_len_cover(df):
    len_list = []
    for title in df['Transcript']:
        wordList = title.split()
        len_list.append(len(wordList))
    return mean(len_list)

In [31]:
per_decade = pd.DataFrame(columns = ['Date', 'Avg. Title Len', 'Avg. Subtitle Len', 'Avg. Words in Whole Cover', 'Most Freq Words', 'Most Freq Word(prepro)'])
for a in range(9, 150, 10):
    df1 = df.iloc[a-9:a]
    t_len = avg_len_title(df1)
    st_len = avg_len_subtitle(df1)
    c_len = avg_len_cover(df1)
    freqwords = freq_words(df1, 5)
    prepro_freq = freq_words_processed(df1, 5)
    period = 1700 + (a-9)
    df2 = pd.DataFrame({'Date': period, 'Avg. Title Len': t_len, 'Avg. Subtitle Len': st_len, 'Avg. Words in Whole Cover': c_len, 'Most Freq Words': [freqwords], 'Most Freq Word(prepro)': [prepro_freq]})
    per_decade = pd.concat([per_decade, df2], ignore_index = True)

9
19
29
39
49
59
69
79
89
99
109
119
129
139
149


In [32]:
per_decade

,Date,Avg. Title Len,Avg. Subtitle Len,Avg. Words in Whole Cover,Most Freq Words,Most Freq Word(prepro)
0,1700,19.222222,13.777778,141.111111,"{'the': 54, 'of': 35, 'and': 32, 'a': 29, 'in'...","{';': 16, 'ee': 15, '.': 14, 'ae': 12, ':': 11}"
1,1710,16.333333,24.222222,144.222222,"{'the': 68, 'of': 48, 'and': 41, 'a': 34, 'in'...","{':': 19, '.': 19, 'ee': 18, 'ae': 15, '4': 9}"
2,1720,12.666667,16.111111,119.000000,"{'the': 36, 'of': 35, 'and': 32, 'in': 12, ':'...","{':': 12, ';': 11, 'de': 9, ',': 6, 'j.': 6}"
3,1730,10.000000,9.333333,159.666667,"{'the': 72, 'of': 51, 'and': 46, 'in': 32, ':'...","{':': 19, '.': 16, 'e': 9, '4': 8, ';': 7}"
4,1740,10.444444,18.222222,155.444444,"{'of': 64, 'the': 64, 'and': 51, 'a': 31, 'in'...","{'ee': 13, ':': 13, '.': 11, ';': 10, 'ae': 9}"
5,1750,24.555556,15.000000,174.888889,"{'the': 65, 'and': 55, 'of': 53, 'a': 39, 'to'...","{';': 25, ':': 22, 'ee': 17, 'ae': 14, '.': 8}"
6,1760,20.444444,1.777778,182.555556,"{'the': 74, 'of': 49, 'a': 40, 'and': 29, 'by'...","{':': 24, '.': 20, ';': 16, '4': 16, 'ee': 14}"
7,1770,13.666667,11.222222,121.333333,"{'and': 33, 'the': 29, 'of': 26, 'a': 25, ':':...","{':': 21, '.': 12, ';': 11, 'f': 9, 'ae': 8}"
8,1780,12.333333,7.888889,215.444444,"{'a': 76, ':': 42, 'of': 36, 'ae': 35, 'the': 34}","{':': 42, 'ae': 35, 'ee': 31, '.': 22, '4': 21}"
9,1790,13.444444,32.222222,131.444444,"{'the': 39, 'and': 35, 'a': 31, 'of': 28, ':':...","{':': 26, ';': 25, '.': 22, 'ae': 15, '3': 11}"
